<a href="https://colab.research.google.com/github/liminovna/SpeechandLanguageProcessingV3_Solutions/blob/main/JurafskyNLP_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3.1. Write out the equation for trigram probability estimation

$P(w_n|w_{n-2}w_{n-1}) = \frac{C(w_{n-2}w_{n-1}w_n)}{C(w_{n-2}w_{n-1})}$

Now write out all the non-zero trigram probabilities for the *I am Sam* corpus

    <s> I am Sam </s>
    <s> Sam I am </s>
    <s> I do not like green eggs and ham </s>

In [29]:
class TrigramModel():
  def __init__(self, corpus):
    # self.unigrams = {}
    self.bigrams = {}
    self.trigrams = {}

    for s in corpus:
      words = s.lower().split()
      # # appending unigrams
      # for w in words:
      #   if w not in self.unigrams.keys():
      #     self.unigrams[w] = 1
      #   else:
      #     self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1
      # appending trigrams
      cntr = 0
      for i in range(len(words)-2):
        cur_trigram = (words[cntr] + ' ' + words[cntr+1] + ' ' + words[cntr+2])
        if cur_trigram not in self.trigrams:
          self.trigrams[cur_trigram] = 1
        else:
          self.trigrams[cur_trigram] += 1
        cntr += 1

  # def add1smoothing_proba(self, s):
  #   s = s.lower()
  #   words = s.split()
  #   return (self.trigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.trigrams[s]/self.bigrams[words[0] + ' ' + words[1]]

In [48]:
corpus = [
    '<s> I am Sam </s>',
    '<s> Sam I am </s>',
    '<s> I do not like green eggs and ham </s>',
]

model = TrigramModel(corpus=corpus)

In [49]:
print('Answer: ')
for t in model.trigrams.keys():
  print(f'P({t}) = {model.unsmoothed_proba(t)}')

Answer: 
P(<s> i am) = 0.5
P(i am sam) = 0.5
P(am sam </s>) = 1.0
P(<s> sam i) = 1.0
P(sam i am) = 1.0
P(i am </s>) = 0.5
P(<s> i do) = 0.5
P(i do not) = 1.0
P(do not like) = 1.0
P(not like green) = 1.0
P(like green eggs) = 1.0
P(green eggs and) = 1.0
P(eggs and ham) = 1.0
P(and ham </s>) = 1.0


## 3.2 Calculate the probability of the sentence *i want chinese food*. Give two probabilities -- unsmoothed and an add-one probability. Assume the additional add-1 smoothed probabilities $P(i|<s>)=0.19$ and $P(</s>|food)=0.40$.

In [46]:
sentence = '<s> I want chinese food <\s>'
unsmoothed_proba = 0.25*0.33*0.0065*0.52*0.68
addone_proba = 0.19*0.21*0.0029*0.52*0.4
print('Answer\n',\
      'Unsmoothed probability:', "{:.5f}".format(unsmoothed_proba), '\n',\
      'Add-one probability:', "{:.7f}".format(addone_proba))

Answer
 Unsmoothed probability: 0.00019 
 Add-one probability: 0.0000241


## 3.3 Which of the two probabilities you computed in the previous exercise is higher, unsmoothed or smoothed? Explain why.

Answer:

The unsmoothed probability is higher because we did not give away any bit of it to the events we have not seen, unlike smoothed probability which is what's left after we've 'shaved off' some of the probability mass.

## 3.4 We are given the following corpus:

    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

Using a bigram language model with add-one smoothing, what is P(Sam|am)? Include \<s> and \<\s> in your counts just like any other token.

In [42]:
class BigramModel():
  def __init__(self, corpus):
    self.unigrams = {}
    self.bigrams = {}

    for s in corpus:
      words = s.split()
      # appending unigrams
      for w in words:
        if w not in self.unigrams.keys():
          self.unigrams[w] = 1
        else:
          self.unigrams[w] += 1
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in self.bigrams:
          self.bigrams[cur_bigram] = 1
        else:
          self.bigrams[cur_bigram] += 1
        cntr += 1

  def add1smoothing_proba(self, s):
    s = s.lower()
    words = s.split()
    return (self.bigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.bigrams[s]/self.unigrams[words[0]]

In [44]:
corpus = [
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',
]

model = BigramModel(corpus=corpus)
print('Answer: ', round(model.add1smoothing_proba('am Sam'), 3))

Answer:  0.214


## 3.5 Suppose we didn't use the end-symbol \<\s>. Train an unsmoothed bigram grammar on the following training corpus without using the end-symbol \<\s>:

    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'

In [ ]:
corpus = [
    '<s> a b',
    '<s> b b',
    '<s> b a',
    '<s> a a'
]

model = BigramModel(corpus=corpus)
model.unsmoothed_proba('a a') + model.unsmoothed_proba('a b') + model.unsmoothed_proba('b b') + model.unsmoothed_proba('b a')

1.0

## 3.6 Suppose we train a trigram language model with add-one smoothing on a given corpus. The corpus contains V word types. Express a formula for estimating $P(w3,w1,w2)$, where $w3$ is a word which follows the bigram $(w1,w2)$, in terms of various n-gram counts and V.

$P_{add-one} (w3|w1w2) = \frac{C(w1w2w3)+1}{C(w1w2)+V_2}$

, where $V_2$ is the number of unique bigrams in the corpus

## 3.7 We are given the following corpus, modified from the one in the chapter:
    '<s> i am sam </s>',
    '<s> sam i am </s>',
    '<s> i am sam </s>',
    '<s> i do not like green eggs and sam </s>',

If we use linear interpolation smoothing between a maximum-likelihood bigram
model and a maximum-likelihood unigram model with $\lambda_1 = \frac{1}{2}$ and $\lambda_2 = \frac{1}{2}$ , what is $P(Sam|am)$? Include \<s> and </s> in your counts just like any other token.

Answer:

$\hat P(Sam|am) = \lambda_1*P(Sam) + \lambda_2*P(am|Sam)$

In [52]:
lambda_1 = 0.5
lambda_2 = 0.5
p_sam = 4/25
p_amsam = 2/3
p_interpolation = lambda_1*p_sam + lambda_2*p_amsam
print('Answer: ', p_interpolation)

Answer:  0.41333333333333333


## 3.8 Write a program to compute unsmoothed unigrams and bigrams.

In [107]:
import random

class NgramExtractor():
  def __init__(self, filename):
    self.filename = filename
    self.corpus = self.read_text()
    self.unigrams = self.extract_unigrams()
    self.bigrams = self.extract_bigrams()

  def read_text(self):
    with open(self.filename) as f:
      lines = f.readlines()

    import re
    corpus = []
    for line in lines:
      line = line.replace('\n', '')
      sentences = re.split('[.?!]\s*', line)
      for s in sentences:
        if s != '':
          corpus.extend(['<s> ' + s.lower().replace(',', '') + ' <\s>'])
    return corpus

  def extract_unigrams(self):

    lst = {}
    for s in self.corpus:
      words = s.split()
      # appending unigrams
      for w in words:
        if w not in lst.keys():
          lst[w] = 1
        else:
          lst[w] += 1
    total = sum(lst.values())
    for index, value in lst.items():
      lst[index] = value/total
    return lst

  def extract_bigrams(self):
    lst = {}
    for s in self.corpus:
      words = s.split()
      # appending bigrams
      cntr = 0
      for i in range(len(words)-1):
        cur_bigram = (words[cntr] + ' ' + words[cntr+1])
        if cur_bigram not in lst:
          lst[cur_bigram] = 1
        else:
          lst[cur_bigram] += 1
        cntr += 1
    total = sum(lst.values())
    for index, value in lst.items():
      lst[index] = value/total
    return lst

  def add1smoothing_proba(self, s):
    s = s.lower()
    words = s.split()
    return (self.bigrams[s] + 1) / (self.unigrams[words[0]]+len(self.unigrams))

  def unsmoothed_proba(self, s):
    s = s.lower()
    words = s.split()
    return self.bigrams[s]/self.unigrams[words[0]]

  def generate_random_sentence(self, length):
    words = list(self.unigrams.keys())
    words.remove('<s>')
    words.remove('<\s>')
    return ' '.join(random.sample(words, length)) + '.'

## 3.9 Run your n-gram program on two different small corpora of your choice (you might use email text or newsgroups). Now compare the statistics of the two corpora. What are the differences in the most common unigrams between the two? How about interesting differences in bigrams?

In [108]:
filename = 'MurderOfTheUniverse.txt' #@param
model1 = NgramExtractor(filename)
model1.corpus[:10]

['<s> as soon as the dust settles you can see <\\s>',
 '<s> a new world in place of where the old one had been <\\s>',
 '<s> your skin is crawling with dry crusted mud <\\s>',
 '<s> and your naked feet are wet in a pool of blood <\\s>',
 '<s> and the whistle of the wind in your ears is so loud <\\s>',
 '<s> that your memories have blown up in a mushroom cloud <\\s>',
 '<s> and as your eyes accommodate <\\s>',
 '<s> there appears by the meadow <\\s>',
 '<s> a brute like a bear with a long dark shadow <\\s>',
 '<s> and you violently shake over what you have seen <\\s>']

In [109]:
import pandas as pd
motu = pd.Series(model1.unigrams).sort_values(ascending=False).drop(index=['<s>', '<\s>'])
motu[:50]

i            0.040387
the          0.033866
and          0.025452
to           0.016828
you          0.015355
a            0.014304
of           0.012200
altered      0.011359
my           0.010938
balrog       0.009886
feel         0.009676
me           0.009466
beast        0.009255
don't        0.007993
am           0.007783
in           0.007783
is           0.007783
your         0.006942
see          0.006521
it           0.006521
vomit        0.006100
oh           0.005259
no           0.005048
alter        0.004628
that         0.004417
with         0.004207
are          0.004207
an           0.004207
from         0.003997
can          0.003786
as           0.003576
like         0.003576
through      0.003155
his          0.003155
want         0.003155
into         0.003155
nonagon      0.002945
for          0.002945
he           0.002735
coffin       0.002735
all          0.002735
will         0.002524
lightning    0.002524
up           0.002314
pain         0.002314
head      

In [100]:
filename = 'NonagonInfinity.txt' #@param
model2 = NgramExtractor(filename)
model2.corpus[:10]

['<s> nonagon infinity opens the door <\\s>',
 '<s> nonagon infinity opens the door <\\s>',
 '<s> wait for the answer to open the door <\\s>',
 '<s> nonagon infinity opens the door <\\s>',
 '<s> one two three <\\s>',
 '<s> loosen up <\\s>',
 '<s> time to drop <\\s>',
 '<s> fuck shit up <\\s>',
 "<s> don't forget about it <\\s>",
 "<s> my coffin's all i see <\\s>"]

In [71]:
nonagon = pd.Series(model2.unigrams).sort_values(ascending=False).drop(index=['<s>', '<\s>'])
nonagon[:50]

the          109
a             51
i             46
my            36
it's          35
beat          33
to            30
i'm           27
door          25
wasp          25
of            24
gamma         24
is            23
knife         22
infinity      22
nonagon       22
only          21
fig           21
and           20
it            18
you           18
miss          18
for           17
mr            16
once          16
opens         15
see           15
up            14
know          14
wah           12
face          12
what          11
when          10
body          10
here           9
can            9
your           9
again          9
all            9
in             9
has            9
universe       8
have           8
invisible      8
down           8
robot          8
same           8
just           8
roll           7
machine        7
dtype: int64

In [48]:
cntr = 0
print('Answer: ')
for t in model.bigrams.keys():
  if cntr < 10:
    # if t.startswith('you '):
    print(f'P({t}) = {model.unsmoothed_proba(t)}')
    cntr +=1

Answer: 
P(<s> as) = 0.008875739644970414
P(as soon) = 0.058823529411764705
P(soon as) = 1.0
P(as the) = 0.35294117647058826
P(the dust) = 0.006211180124223602
P(dust settles) = 0.5
P(settles you) = 1.0
P(you can) = 0.0273972602739726
P(can see) = 0.3888888888888889
P(see <\s>) = 0.22580645161290322


Answer:

Not really the best or even suitable material for this excercise but still better than nothing. Initially was planning to use happy new year messages published by some independent russian media in 2020, 2021 vs. 2022, 2023, because i thought they would show the shift of the focus from the coronavirus in 2020-2021 to the war in 2022-2023. But sadly i didn't find any media that published such messages for 4 years in a row. So instead settled for lyrics from two King Gizzard and the Lizzard Wizard albums -- Murder of the Universe and Nonagon Infinity.

Both showed the highest frequencies for the articles 'the' and 'a' and pronouns like 'i', 'me' and 'my'. In the first album, among most used words are those associated with repulsiveness and evil, such as 'beast', 'balrog', 'vomit', 'pain', 'coffin' etc. which goes hand in hand with the album's narrative.

## 3.10 Add an option to your program to generate random sentences.

In [101]:
# generate a random sentence of 10 words
model1.generate_random_sentence(10)

'swathe deepest heave sabre life lugubrious slip a rotten soggy.'

In [97]:
# generate a random sentence of 10 words
model2.generate_random_sentence(10)

'limbs dream dueling rubber obliteration while winged of aflame fig.'

## 3.11 Add an option to your program to compute the perplexity of a test set.